# Toronto Neighbourhood

[FIRST SUBMISSION](#FIRST-SUBMISSION)  
[SECOND SUBMISSION](#SECOND-SUBMISSION)  
[THIRD SUBMISSION](#THIRD-SUBMISSION)  

## FIRST SUBMISSION

#### First, intall and import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!pip install geopy

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [12]:
from pandas.io.html import read_html

#### Now, read the Canada postal code Wiki web page 

In [13]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

wikitables = read_html(page,  attrs={"class":"wikitable"})

print ("Extracted {num} wikitables".format(num=len(wikitables)))

Extracted 1 wikitables


#### Import the Toronto data into pandas DataFrame

In [14]:
toronto_df = pd.DataFrame(wikitables[0])
toronto_df.head()

Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront

#### Let's check DataFrame's dimms

In [15]:
toronto_df.shape

(288, 3)

#### We have to keep only valid rows, ignore the 'Not assigned' Borough values

In [16]:
toronto_df = toronto_df[toronto_df['Borough']!= 'Not assigned']
toronto_df.head()

Postcode           Borough     Neighbourhood
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
5      M5A  Downtown Toronto       Regent Park
6      M6A        North York  Lawrence Heights

#### Now, group DataFrame by _Postcode_ and _Borough_ to join the multiple _Neighbourhood_ values into single one

In [17]:
toronto_df_group = toronto_df.groupby(['Postcode', 'Borough'])['Neighbourhood'].agg(', '.join).reset_index(name='Neighbourhood')

In [18]:
toronto_df_group.head(10)

Postcode      Borough                                    Neighbourhood
0      M1B  Scarborough                                   Rouge, Malvern
1      M1C  Scarborough           Highland Creek, Rouge Hill, Port Union
2      M1E  Scarborough                Guildwood, Morningside, West Hill
3      M1G  Scarborough                                           Woburn
4      M1H  Scarborough                                        Cedarbrae
5      M1J  Scarborough                              Scarborough Village
6      M1K  Scarborough      East Birchmount Park, Ionview, Kennedy Park
7      M1L  Scarborough                  Clairlea, Golden Mile, Oakridge
8      M1M  Scarborough  Cliffcrest, Cliffside, Scarborough Village West
9      M1N  Scarborough                      Birch Cliff, Cliffside West

#### Finnaly, check again

In [19]:
toronto_df_group.shape

(103, 3)

#### Chenge _Not assigned_ value on M7A Postcode with _Borough's_ value

In [20]:
for index, value in toronto_df_group.iterrows():
    if value['Neighbourhood'] == 'Not assigned':
        new_value = value['Borough']
        value['Neighbourhood'] = new_value

#### Let's see the M7A postcode

In [21]:
toronto_df_group[toronto_df_group['Postcode'] == 'M7A']

Postcode       Borough Neighbourhood
85      M7A  Queen's Park  Queen's Park

#### Everything seems allright till now

## SECOND SUBMISSION

#### Now,  we are trying to request Toronto's location by Fourquesst API

In [45]:
CLIENT_ID = 'ZISPWCOUB0LBELFZ1MZVW5PATP4J4SRMYNJ5WRYEWMIJXD3P' # your Foursquare ID ZISPWCOUB0LBELFZ1MZVW5PATP4J4SRMYNJ5WRYEWMIJXD3P
CLIENT_SECRET = '1V4G4GL3Z4JJSRCPRLRBINCEINQFHBHFBPTRUCGRQY1NZSKE' # your Foursquare Secret
VERSION = '20191030'  #20180604
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZISPWCOUB0LBELFZ1MZVW5PATP4J4SRMYNJ5WRYEWMIJXD3P
CLIENT_SECRET:1V4G4GL3Z4JJSRCPRLRBINCEINQFHBHFBPTRUCGRQY1NZSKE


In [80]:
address = '393 University Ave, Toronto, ON M5G 1E6, Canada'

geolocator = Nominatim(user_agent="foursquare_agent", timeout=15)

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The coordinates of Toronto are: {}, {}".format(latitude, longitude))

The coordinates of Toronto are: 43.6541752, -79.3876027


In [81]:
for index, row in toronto_df_group.iterrows():
    address = row['Borough']
    geolocator = Nominatim(user_agent="foursquare_agent", timeout=35)
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    toronto_df_group['latitude'] = latitude
    toronto_df_group['longitude'] = longitude

In [83]:
toronto_df_group.head(20)

Postcode      Borough                                      Neighbourhood  \
0       M1B  Scarborough                                     Rouge, Malvern   
1       M1C  Scarborough             Highland Creek, Rouge Hill, Port Union   
2       M1E  Scarborough                  Guildwood, Morningside, West Hill   
3       M1G  Scarborough                                             Woburn   
4       M1H  Scarborough                                          Cedarbrae   
5       M1J  Scarborough                                Scarborough Village   
6       M1K  Scarborough        East Birchmount Park, Ionview, Kennedy Park   
7       M1L  Scarborough                    Clairlea, Golden Mile, Oakridge   
8       M1M  Scarborough    Cliffcrest, Cliffside, Scarborough Village West   
9       M1N  Scarborough                        Birch Cliff, Cliffside West   
10      M1P  Scarborough  Dorset Park, Scarborough Town Centre, Wexford ...   
11      M1R  Scarborough                                  Maryvale, Wexford   
12      M1S  Scarborough                                          Agincourt   
13      M1T  Scarborough            Clarks Corners, Sullivan, Tam O'Shanter   
14      M1V  Scarborough  Agincourt North, L'Amoreaux East, Milliken, St...   
15      M1W  Scarborough                                    L'Amoreaux West   
16      M1X  Scarborough                                        Upper Rouge   
17      M2H   North York                                  Hillcrest Village   
18      M2J   North York                       Fairview, Henry Farm, Oriole   
19      M2K   North York                                    Bayview Village   

     Latitude  Longitude   latidude  longitude  
0   43.806686 -79.194353  43.671459 -79.552492  
1   43.784535 -79.160497  43.671459 -79.552492  
2   43.763573 -79.188711  43.671459 -79.552492  
3   43.770992 -79.216917  43.671459 -79.552492  
4   43.773136 -79.239476  43.671459 -79.552492  
5   43.744734 -79.239476  43.671459 -79.552492  
6   43.727929 -79.262029  43.671459 -79.552492  
7   43.711112 -79.284577  43.671459 -79.552492  
8   43.716316 -79.239476  43.671459 -79.552492  
9   43.692657 -79.264848  43.671459 -79.552492  
10  43.757410 -79.273304  43.671459 -79.552492  
11  43.750072 -79.295849  43.671459 -79.552492  
12  43.794200 -79.262029  43.671459 -79.552492  
13  43.781638 -79.304302  43.671459 -79.552492  
14  43.815252 -79.284577  43.671459 -79.552492  
15  43.799525 -79.318389  43.671459 -79.552492  
16  43.836125 -79.205636  43.671459 -79.552492  
17  43.803762 -79.363452  43.671459 -79.552492  
18  43.778517 -79.346556  43.671459 -79.552492  
19  43.786947 -79.385975  43.671459 -79.552492

#### It seems not working...:((  
remove 'latitude', 'longitude' columns

In [86]:
toronto_df_group = toronto_df_group.drop(columns=['latitude', 'longitude'])
toronto_df_group.head()

Postcode      Borough                           Neighbourhood   Latitude  \
0      M1B  Scarborough                          Rouge, Malvern  43.806686   
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2      M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3      M1G  Scarborough                                  Woburn  43.770992   
4      M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

#### We have to mannualy download and import into Pandas df the Toronto's coordonates

In [88]:
toronto_coord = pd.read_csv('Geospatial_Coordinates.csv')
toronto_coord.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

#### Let's check the dataframe's shapes

In [89]:
print(toronto_df_group.shape, toronto_coord.shape,
    toronto_df_group.shape==toronto_coord.shape)

(103, 5) (103, 3) False


#### Now, let's copy the Latitude and Longitude columns to our DataFrame

In [90]:
(toronto_df_group['Latitude'], toronto_df_group['Longitude']) = (toronto_coord['Latitude'], toronto_coord['Longitude'])

In [91]:
toronto_df_group.head()

Postcode      Borough                           Neighbourhood   Latitude  \
0      M1B  Scarborough                          Rouge, Malvern  43.806686   
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2      M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3      M1G  Scarborough                                  Woburn  43.770992   
4      M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

## THIRD SUBMISSION

### Let's plot neighborhoods's map

In [93]:
import folium

In [95]:
map_of_toronto = folium.Map(location = [latitude, longitude], zoom_start = 11)

for borough, neighborhood, latitude, longitude in zip(toronto_df_group['Borough'], 
                                                      toronto_df_group['Neighbourhood'], 
                                                      toronto_df_group['Latitude'], 
                                                      toronto_df_group['Longitude']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [latitude, longitude],
        radius = 5,
        popup = label,
        color = 'green',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_of_toronto)  
    
map_of_toronto

## Analysing first neighborhood

I'll then check out the first neighborhood in the selected dataset.

In [98]:
print("The neighborhood: {}".format(toronto_df_group.loc[0, 'Neighbourhood']))

The neighborhood: Rouge, Malvern


In [99]:
neigh_name = toronto_df_group.loc[0, 'Neighbourhood']
neigh_lat = toronto_df_group.loc[0, 'Latitude']
neigh_long = toronto_df_group.loc[0, 'Longitude']
print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_lat, 
                                                               neigh_long))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


#### Let's explore top 100 venues located in **Rouge, Malvern** within a radius of 500m.

In [ ]:
RADIUS = 1500
LIMIT = 100
url = ('https://api.foursquare.com/v2/venues/explore?client_id={}'
       '&client_secret={}&v={}&ll={},{}&radius={}&limit={}').format(CLIENT_ID, 
                                                                  CLIENT_SECRET, 
                                                                  VERSION, 
                                                                  neigh_lat, 
                                                                  neigh_long, 
                                                                  RADIUS,
                                                                  LIMIT)
result = requests.get(url).json()
result

#### Let's extract Category type

In [107]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [111]:
venues = result['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

# Filter the columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# Filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)

# Clean all column names
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(30)

name                     categories        lat  \
0         Images Salon & Spa                            Spa  43.802283   
1          Canadiana exhibit                    Zoo Exhibit  43.817962   
2                       LCBO                   Liquor Store  43.796671   
3                    Wendy's           Fast Food Restaurant  43.802008   
4             Caribbean Wave           Caribbean Restaurant  43.798558   
5                    Wendy's           Fast Food Restaurant  43.807448   
6        Staples Morningside  Paper / Office Supplies Store  43.800285   
7                   Harvey's           Fast Food Restaurant  43.800106   
8                Pizza Pizza                    Pizza Place  43.806613   
9                Tim Hortons                    Coffee Shop  43.802000   
10              Boston Pizza                    Pizza Place  43.796470   
11        Tim Hortons / Esso                    Coffee Shop  43.801863   
12  Charley's Exotic Cuisine             Chinese Restaurant  43.800982   
13        Fusion Supermarket        Fruit & Vegetable Store  43.800528   
14             What a Bakery                         Bakery  43.798888   
15     Sally's Beauty Supply                 Cosmetics Shop  43.798690   
16                  EB Games               Video Game Store  43.798753   
17          Ontrack Services     Financial or Legal Service  43.816399   
18      Grizzly Bear Exhibit                    Zoo Exhibit  43.817031   
19                  Reitmans                  Women's Store  43.797893   
20           Cheetah Exhibit           Other Great Outdoors  43.817588   
21                McDonald's           Fast Food Restaurant  43.798261   
22        simba safari lodge                    Zoo Exhibit  43.817305   
23                  Kelsey's            American Restaurant  43.796686   
24  White Rhinoceros exhibit                    Zoo Exhibit  43.817434   
25             Zebra Exhibit                    Zoo Exhibit  43.818045   
26             Kesho Park HQ                    Zoo Exhibit  43.817257   
27      Olive Baboon Exhibit                    Zoo Exhibit  43.818545   
28                   Walmart                  Big Box Store  43.797863   
29      Hippopotamus Exhibit                    Zoo Exhibit  43.817179   

          lng  
0  -79.198565  
1  -79.193374  
2  -79.204586  
3  -79.198080  
4  -79.195777  
5  -79.199056  
6  -79.196607  
7  -79.198258  
8  -79.178445  
9  -79.198169  
10 -79.204245  
11 -79.199296  
12 -79.200233  
13 -79.200064  
14 -79.196473  
15 -79.201516  
16 -79.201684  
17 -79.199157  
18 -79.193458  
19 -79.203138  
20 -79.187206  
21 -79.201449  
22 -79.185904  
23 -79.202631  
24 -79.185474  
25 -79.187056  
26 -79.184667  
27 -79.187238  
28 -79.200654  
29 -79.183676

In [112]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

33 venues were returned by Foursquare.


## Get venues for all neighborhoods

In [113]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [114]:
list_of_venues = getNearbyVenues(toronto_df_group['Neighbourhood'],
                                 toronto_df_group['Latitude'],
                                 toronto_df_group['Longitude'],
                                 RADIUS)

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [115]:
list_of_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Adelaide, King, Richmond                                              100   
Agincourt                                                              62   
Agincourt North, L'Amoreaux East, Milliken, Ste...                     69   
Albion Gardens, Beaumond Heights, Humbergate, J...                     27   
Alderwood, Long Branch                                                 44   
Bathurst Manor, Downsview North, Wilson Heights                        39   
Bayview Village                                                        12   
Bedford Park, Lawrence Manor East                                      74   
Berczy Park                                                           100   
Birch Cliff, Cliffside West                                            15   
Bloordale Gardens, Eringate, Markland Wood, Old...                     41   
Brockton, Exhibition Place, Parkdale Village                          100   
Business Reply Mail Processing Centre 969 Eastern                     100   
CFB Toronto, Downsview East                                            30   
CN Tower, Bathurst Quay, Island airport, Harbou...                     70   
Cabbagetown, St. James Town                                           100   
Caledonia-Fairbanks                                                    76   
Canada Post Gateway Processing Centre                                  61   
Cedarbrae                                                              59   
Central Bay Street                                                    100   
Chinatown, Grange Park, Kensington Market                             100   
Christie                                                              100   
Church and Wellesley                                                  100   
Clairlea, Golden Mile, Oakridge                                        37   
Clarks Corners, Sullivan, Tam O'Shanter                                44   
Cliffcrest, Cliffside, Scarborough Village West                        40   
Cloverdale, Islington, Martin Grove, Princess G...                     19   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                            100   
Davisville North                                                      100   
Deer Park, Forest Hill SE, Rathnelly, South Hil...                    100   
Del Ray, Keelesdale, Mount Dennis, Silverthorn                         39   
Design Exchange, Toronto Dominion Centre                              100   
Don Mills North                                                        90   
Dorset Park, Scarborough Town Centre, Wexford H...                     58   
Dovercourt Village, Dufferin                                          100   
Downsview Central                                                      34   
Downsview Northwest                                                    34   
Downsview West                                                         11   
Downsview, North Park, Upwood Park                                     30   
East Birchmount Park, Ionview, Kennedy Park                            40   
East Toronto                                                           87   
Emery, Humberlea                                                       16   
Fairview, Henry Farm, Oriole                                           62   
First Canadian Place, Underground city                                100   
Flemingdon Park, Don Mills South                                       83   
Forest Hill North, Forest Hill West                                    97   
Glencairn                                                              57   
Guildwood, Morningside, West Hill                                      36   
Harbord, University of Toronto                                        100   
Harbourfront East, Toronto Islands, Union Statio

## Neighborhood analysis

In [116]:
toronto_onehot = pd.get_dummies(list_of_venues[['Venue Category']], prefix = "", prefix_sep = "")
toronto_onehot = pd.concat([list_of_venues['Neighborhood'], toronto_onehot.drop(['Neighborhood'], axis = 1)], axis = 1)
toronto_onehot.head()

Neighborhood  Accessories Store  Afghan Restaurant  African Restaurant  \
0  Rouge, Malvern                  0                  0                   0   
1  Rouge, Malvern                  0                  0                   0   
2  Rouge, Malvern                  0                  0                   0   
3  Rouge, Malvern                  0                  0                   0   
4  Rouge, Malvern                  0                  0                   0   

   Airport  Airport Lounge  American Restaurant  Amphitheater  Animal Shelter  \
0        0               0                    0             0               0   
1        0               0                    0             0               0   
2        0               0                    0             0               0   
3        0               0                    0             0               0   
4        0               0                    0             0               0   

   Antique Shop  Aquarium  Arcade  Argentinian Restaurant  Art Gallery  \
0             0         0       0                       0            0   
1             0         0       0                       0            0   
2             0         0       0                       0            0   
3             0         0       0                       0            0   
4             0         0       0                       0            0   

   Art Museum  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0           0                    0                 0                   0   
1           0                    0                 0                   0   
2           0                    0                 0                   0   
3           0                    0                 0                   0   
4           0                    0                 0                   0   

   Auto Dealership  Automotive Shop  BBQ Joint  Baby Store  Badminton Court  \
0                0                0          0           0                0   
1                0                0          0           0                0   
2                0                0          0           0                0   
3                0                0          0           0                0   
4                0                0          0           0                0   

   Bagel Shop  Bakery  Bank  Bar  Baseball Field  Baseball Stadium  \
0           0       0     0    0               0                 0   
1           0       0     0    0               0                 0   
2           0       0     0    0               0                 0   
3           0       0     0    0               0                 0   
4           0       0     0    0               0                 0   

   Basketball Court  Basketball Stadium  Beach  Beach Bar  Beer Bar  \
0                 0                   0      0          0         0   
1                 0                   0      0          0         0   
2                 0                   0      0          0         0   
3                 0                   0      0          0         0   
4                 0                   0      0          0         0   

   Beer Store  Belgian Restaurant  Big Box Store  Bike Shop  Bistro  \
0           0                   0              0          0       0   
1           0                   0              0          0       0   
2           0                   0              0          0       0   
3           0                   0              0          0       0   
4           0                   0              0          0       0   

   Boat or Ferry  Bookstore  Botanical Garden  Boutique  Bowling Alley  \
0              0          0                 0         0              0   
1              0          0                 0         0              0   
2              0          0                 0         0              0   
3              0          0                 0         0              0   
4              0     

In [117]:
toronto_onehot.shape

(6848, 351)

#### The mean frequency

In [118]:
toronto_grouped = toronto_onehot.groupby(['Neighborhood']).mean().reset_index()
toronto_grouped

Neighborhood  Accessories Store  \
0                             Adelaide, King, Richmond               0.00   
1                                            Agincourt               0.00   
2    Agincourt North, L'Amoreaux East, Milliken, St...               0.00   
3    Albion Gardens, Beaumond Heights, Humbergate, ...               0.00   
4                               Alderwood, Long Branch               0.00   
5      Bathurst Manor, Downsview North, Wilson Heights               0.00   
6                                      Bayview Village               0.00   
7                    Bedford Park, Lawrence Manor East               0.00   
8                                          Berczy Park               0.00   
9                          Birch Cliff, Cliffside West               0.00   
10   Bloordale Gardens, Eringate, Markland Wood, Ol...               0.00   
11        Brockton, Exhibition Place, Parkdale Village               0.00   
12   Business Reply Mail Processing Centre 969 Eastern               0.00   
13                         CFB Toronto, Downsview East               0.00   
14   CN Tower, Bathurst Quay, Island airport, Harbo...               0.00   
15                         Cabbagetown, St. James Town               0.00   
16                                 Caledonia-Fairbanks               0.00   
17               Canada Post Gateway Processing Centre               0.00   
18                                           Cedarbrae               0.00   
19                                  Central Bay Street               0.00   
20           Chinatown, Grange Park, Kensington Market               0.00   
21                                            Christie               0.00   
22                                Church and Wellesley               0.00   
23                     Clairlea, Golden Mile, Oakridge               0.00   
24             Clarks Corners, Sullivan, Tam O'Shanter               0.00   
25     Cliffcrest, Cliffside, Scarborough Village West               0.00   
26   Cloverdale, Islington, Martin Grove, Princess ...               0.00   
27                      Commerce Court, Victoria Hotel               0.00   
28                                          Davisville               0.00   
29                                    Davisville North               0.00   
30   Deer Park, Forest Hill SE, Rathnelly, South Hi...               0.00   
31      Del Ray, Keelesdale, Mount Dennis, Silverthorn               0.00   
32            Design Exchange, Toronto Dominion Centre               0.00   
33                                     Don Mills North               0.00   
34   Dorset Park, Scarborough Town Centre, Wexford ...               0.00   
35                        Dovercourt Village, Dufferin               0.00   
36                                   Downsview Central               0.00   
37                                 Downsview Northwest               0.00   
38                                      Downsview West               0.00   
39                  Downsview, North Park, Upwood Park               0.00   
40         East Birchmount Park, Ionview, Kennedy Park               0.00   
41                                        East Toronto               0.00   
42                                    Emery, Humberlea               0.00   
43                        Fairview, Henry Farm, Oriole               0.00   
44              First Canadian Place, Underground city               0.00   
45                    Flemingdon Park, Don Mills South               0.00   
46                 Forest Hill North, Forest Hill West               0.00   
47                                           Glencairn               0.00   
48                   Guildwood, Morningside, West Hill               0.00   
49                      Harbord, University of Toronto               0.00   
50   Harbourfront East, Toronto Islands, Union Station               0.00   
51                           Harbourfr

In [119]:
toronto_grouped.shape

(103, 351)

#### Let's plot the top 5 common venues in each neighborhood.

In [120]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop = True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0        Hotel  0.07
1  Coffee Shop  0.07
2         Café  0.06
3      Theater  0.04
4    Gastropub  0.03


----Agincourt----
                venue  freq
0  Chinese Restaurant  0.19
1              Bakery  0.05
2         Coffee Shop  0.05
3       Shopping Mall  0.05
4      Breakfast Spot  0.03


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0  Chinese Restaurant  0.16
1         Coffee Shop  0.06
2            Pharmacy  0.06
3     Bubble Tea Shop  0.06
4        Dessert Shop  0.04


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0  Fast Food Restaurant  0.19
1           Coffee Shop  0.15
2         Grocery Store  0.11
3           Pizza Place  0.11
4      Greek Restaurant  0.04


----Alderwood, Long Branch----
           venue  freq
0           Park  0.07
1   Burger Joint  0.05
2     Restaurant

In [121]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Let's get the top 10 venues for each location.

In [122]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                           Adelaide, King, Richmond           Coffee Shop   
1                                          Agincourt    Chinese Restaurant   
2  Agincourt North, L'Amoreaux East, Milliken, St...    Chinese Restaurant   
3  Albion Gardens, Beaumond Heights, Humbergate, ...  Fast Food Restaurant   
4                             Alderwood, Long Branch                  Park   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                 Hotel                  Café               Theater   
1           Coffee Shop         Shopping Mall                Bakery   
2              Pharmacy           Coffee Shop       Bubble Tea Shop   
3           Coffee Shop           Pizza Place         Grocery Store   
4         Grocery Store           Pizza Place              Pharmacy   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0           Pizza Place            Restaurant             Gastropub   
1  Caribbean Restaurant  Gym / Fitness Center        Breakfast Spot   
2     Korean Restaurant             BBQ Joint                Bakery   
3        Sandwich Place        Clothing Store            Beer Store   
4           Coffee Shop                  Café          Burger Joint   

  8th Most Common Venue 9th Most Common Venue  10th Most Common Venue  
0              Beer Bar   Sporting Goods Shop  Furniture / Home Store  
1  Cantonese Restaurant    Seafood Restaurant     Rental Car Location  
2           Pizza Place          Dessert Shop            Noodle House  
3   Fried Chicken Joint      Greek Restaurant       Electronics Store  
4      Toy / Game Store            Restaurant     Sporting Goods Shop

## Clustering

In [125]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# Run K-Means clustering
kMeans = KMeans(n_clusters = 5, random_state = 0).fit(toronto_grouped_clustering)

In [133]:
neighborhoods_venues_sorted.head()

Cluster Labels                                       Neighborhood  \
0               1                           Adelaide, King, Richmond   
1               4                                          Agincourt   
2               4  Agincourt North, L'Amoreaux East, Milliken, St...   
3               0  Albion Gardens, Beaumond Heights, Humbergate, ...   
4               4                             Alderwood, Long Branch   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0           Coffee Shop                 Hotel                  Café   
1    Chinese Restaurant           Coffee Shop         Shopping Mall   
2    Chinese Restaurant              Pharmacy           Coffee Shop   
3  Fast Food Restaurant           Coffee Shop           Pizza Place   
4                  Park         Grocery Store           Pizza Place   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0               Theater           Pizza Place            Restaurant   
1                Bakery  Caribbean Restaurant  Gym / Fitness Center   
2       Bubble Tea Shop     Korean Restaurant             BBQ Joint   
3         Grocery Store        Sandwich Place        Clothing Store   
4              Pharmacy           Coffee Shop                  Café   

  7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0             Gastropub              Beer Bar   Sporting Goods Shop   
1        Breakfast Spot  Cantonese Restaurant    Seafood Restaurant   
2                Bakery           Pizza Place          Dessert Shop   
3            Beer Store   Fried Chicken Joint      Greek Restaurant   
4          Burger Joint      Toy / Game Store            Restaurant   

   10th Most Common Venue  
0  Furniture / Home Store  
1     Rental Car Location  
2            Noodle House  
3       Electronics Store  
4     Sporting Goods Shop

In [137]:
# Add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kMeans.labels_)

# Combine the data
toronto_merged = toronto_df_group.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighbourhood')
toronto_merged.head()

Postcode      Borough                           Neighbourhood   Latitude  \
0      M1B  Scarborough                          Rouge, Malvern  43.806686   
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2      M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3      M1G  Scarborough                                  Woburn  43.770992   
4      M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.194353               0           Zoo Exhibit  Fast Food Restaurant   
1 -79.160497               4        Breakfast Spot                  Park   
2 -79.188711               0           Pizza Place        Breakfast Spot   
3 -79.216917               0           Coffee Shop              Pharmacy   
4 -79.239476               4           Coffee Shop        Clothing Store   

  3rd Most Common Venue 4th Most Common Venue          5th Most Common Venue  \
0           Pizza Place           Coffee Shop  Paper / Office Supplies Store   
1          Burger Joint            Playground                  Grocery Store   
2           Coffee Shop  Fast Food Restaurant                  Grocery Store   
3           Pizza Place        Clothing Store                 Discount Store   
4        Sandwich Place     Indian Restaurant                    Wings Joint   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0                   Spa        Cosmetics Shop         Movie Theater   
1                   Gym  Gym / Fitness Center    Italian Restaurant   
2            Beer Store   Fried Chicken Joint           Supermarket   
3     Indian Restaurant                  Park        Sandwich Place   
4                Bakery              Pharmacy            Restaurant   

  9th Most Common Venue   10th Most Common Venue  
0         Big Box Store  Fruit & Vegetable Store  
1   Empanada Restaurant                Drugstore  
2          Liquor Store        Food & Drink Shop  
3  Fast Food Restaurant               Beer Store  
4                  Bank       Italian Restaurant

In [140]:
# create map
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for neighborhood, cluster, latitude, longitude in zip(toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels'], toronto_merged['Latitude'], toronto_merged['Longitude']):
    label = folium.Popup(str(neighborhood) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [latitude, longitude],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

## Examining the clusters

Let's try and see each cluster and the most common venue among each.

In [143]:
# For Cluster 0
result_0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print("For cluster {}, the distribution of venues is as:\n{}".format(0, result_0['1st Most Common Venue'].value_counts()))

For cluster 0, the distribution of venues is as:
Coffee Shop              10
Fast Food Restaurant      5
Pizza Place               4
Vietnamese Restaurant     1
Sandwich Place            1
Zoo Exhibit               1
Chinese Restaurant        1
Park                      1
Name: 1st Most Common Venue, dtype: int64


In [144]:
# For Cluster 1
result_1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print("For cluster {}, the distribution of venues is as:\n{}".format(1, result_1['1st Most Common Venue'].value_counts()))


For cluster 1, the distribution of venues is as:
Coffee Shop         13
Café                 8
Hotel                3
Greek Restaurant     2
Bar                  1
Name: 1st Most Common Venue, dtype: int64


In [145]:
# For Cluster 2
result_2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print("For cluster {}, the distribution of venues is as:\n{}".format(2, result_2['1st Most Common Venue'].value_counts()))


For cluster 2, the distribution of venues is as:
Park              8
Hotel             1
Baseball Field    1
Name: 1st Most Common Venue, dtype: int64


In [146]:
# For Cluster 3
result_3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print("For cluster {}, the distribution of venues is as:\n{}".format(3, result_3['1st Most Common Venue'].value_counts()))

For cluster 3, the distribution of venues is as:
Farm    1
Name: 1st Most Common Venue, dtype: int64


In [147]:
# For Cluster 4
result_4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print("For cluster {}, the distribution of venues is as:\n{}".format(4, result_4['1st Most Common Venue'].value_counts()))

For cluster 4, the distribution of venues is as:
Coffee Shop                  24
Park                          3
Pharmacy                      2
Korean Restaurant             2
Chinese Restaurant            2
Italian Restaurant            2
Clothing Store                1
Athletics & Sports            1
Furniture / Home Store        1
Golf Course                   1
Breakfast Spot                1
Middle Eastern Restaurant     1
Name: 1st Most Common Venue, dtype: int64


# COFFEE SHOP!!!